In [ ]:
!pip install facenet-pytorch
!pip install torchvision

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import zipfile
# import os

# zip_path = "/content/drive/MyDrive/DM daisee/DAiSEE_Frames_Every3s.zip"
# extract_path = "/content/drive/MyDrive/DM daisee/DAiSEE_Frames_Every3s"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

In [ ]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch
import os
import numpy as np

# MTCNN config
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=224, margin=0, keep_all=False, device=device)

# Paths
input_base = "/content/drive/MyDrive/DM daisee/DAiSEE_Frames_Every3s/DAiSEE_Frames_Every3s"
output_base = "/content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces"
os.makedirs(output_base, exist_ok=True)

# Loop through splits
for split in ['Train', 'Test', 'Validation']:
    split_path = os.path.join(input_base, split)
    split_output_path = os.path.join(output_base, split)
    os.makedirs(split_output_path, exist_ok=True)

    for group_id in os.listdir(split_path):
        group_path = os.path.join(split_path, group_id)
        if not os.path.isdir(group_path):
            continue

        for video_id in os.listdir(group_path):
            video_path = os.path.join(group_path, video_id)
            if not os.path.isdir(video_path):
                continue

            output_video_path = os.path.join(split_output_path, group_id, video_id)
            os.makedirs(output_video_path, exist_ok=True)

            for image_file in os.listdir(video_path):
                if not image_file.lower().endswith((".jpg", ".jpeg", ".png")):
                    continue

                image_path = os.path.join(video_path, image_file)
                save_path = os.path.join(output_video_path, image_file)

                # Skip if already processed
                if os.path.exists(save_path):
                    continue

                try:
                    img = Image.open(image_path).convert("RGB")
                    img = img.resize((640, 480))  # boost face detection

                    face = mtcnn(img)

                    if face is not None:
                        face_img = face.permute(1, 2, 0).cpu().numpy()
                        face_img = np.clip(face_img * 255, 0, 255).astype("uint8")
                        face_pil = Image.fromarray(face_img)
                        face_pil.save(save_path)
                        print(f"Saved: {save_path}")
                    else:
                        print(f" No face in: {image_path}")
                except Exception as e:
                    print(f"Error: {image_path} — {str(e)}")


Streaming output truncated to the last 5000 lines.
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202053/image_000.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202053/image_001.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202053/image_002.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202053/image_003.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202054/image_000.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202054/image_001.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202054/image_002.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202054/image_003.jpg
✅ Saved: /content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation/410020/4100202055/image_000.jpg
✅ Sa

In [ ]:
#Testing code for just one

# from facenet_pytorch import MTCNN
# from PIL import Image
# import torch
# import os
# import numpy as np

# # Setup
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# mtcnn = MTCNN(image_size=224, margin=0, keep_all=False, device=device)

# # Test image path (change if needed)
# test_image_path = "/content/drive/MyDrive/DM daisee/DAiSEE_Frames_Every3s/DAiSEE_Frames_Every3s/Train/110001/1100011002/image_000.jpg"
# test_save_path = "/content/drive/MyDrive/DM daisee/DAiSEE_Frames_Every3s/face_test_output.jpg"

# # Run detection
# img = Image.open(test_image_path).convert("RGB")

# face = mtcnn(img)

# if face is not None:
#     print("Face detected!")
#     face_img = face.permute(1, 2, 0).cpu().numpy()
#     face_img = np.clip(face_img * 255, 0, 255).astype("uint8")
#     face_pil = Image.fromarray(face_img)
#     face_pil.save(test_save_path)
#     print(f"Saved cropped face to: {test_save_path}")
# else:
#     print("No face found.")

✅ Face detected!
Saved cropped face to: /content/drive/MyDrive/DM daisee/DAiSEE_Frames_Every3s/face_test_output.jpg


In [ ]:
import os
import shutil
import pandas as pd

# Load the full label CSV
label_df = pd.read_csv("/content/drive/MyDrive/DM daisee/AllLabels.csv")
label_df.columns = label_df.columns.str.strip()

# Source cropped images
cropped_base = "/content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Train"

# Output path for sample
sample_base = "/content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100"
os.makedirs(sample_base, exist_ok=True)

# Output path for reduced CSV
csv_output = "/content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100_Labels.csv"

# Get first 100 valid samples
selected_rows = []
image_count = 0

for _, row in label_df.iterrows():
    if image_count >= 1000:
        break

    clip_id = row['ClipID'].replace('.avi', '')
    group_id = clip_id[:6]
    video_folder = os.path.join(cropped_base, group_id, clip_id)

    if os.path.exists(video_folder):
        image_files = [f for f in os.listdir(video_folder) if f.endswith('.jpg')]
        if not image_files:
            continue

        # Create output folder: 1100011002/
        out_folder = os.path.join(sample_base, clip_id)
        os.makedirs(out_folder, exist_ok=True)

        for img_file in image_files:
            src_img = os.path.join(video_folder, img_file)
            dst_img = os.path.join(out_folder, img_file)
            shutil.copy(src_img, dst_img)

            # Update CSV row with new ClipID (clip_frame.jpg)
            new_row = row.copy()
            new_row['ClipID'] = f"{clip_id}_{img_file}"
            selected_rows.append(new_row)
            image_count += 1

            if image_count >= 1000:
                break

# Save reduced label CSV
reduced_df = pd.DataFrame(selected_rows)
reduced_df.to_csv(csv_output, index=False)

print(f"100 image dataset created at: {sample_base}")
print(f" Labels saved at: {csv_output}")


✅ 100 image dataset created at: /content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100
📄 Labels saved at: /content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100_Labels.csv


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

class DAiSEEMultiLabelDataset(Dataset):
    def __init__(self, csv_path, image_root, transform=None):
        self.data = pd.read_csv(csv_path)
        self.data.columns = self.data.columns.str.strip()
        self.image_root = image_root
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        clip_id = row['ClipID']
        folder, image = clip_id.split('_', 1)
        img_path = os.path.join(self.image_root, folder, image)

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        labels = torch.tensor([
            row['Boredom'],
            row['Engagement'],
            row['Confusion'],
            row['Frustration']
        ], dtype=torch.long)

        return image, labels


In [ ]:
class MultiHeadResNet(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        self.heads = nn.ModuleList([nn.Linear(512, 4) for _ in range(4)])  # 4 outputs

    def forward(self, x):
        x = self.base(x)
        return [head(x) for head in self.heads]


In [ ]:
# Paths
csv_path = "/content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100_Labels.csv"
image_root = "/content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100"

# Dataset & Loader
dataset = DAiSEEMultiLabelDataset(csv_path, image_root, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Device & model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()
model = MultiHeadResNet(resnet).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd

#  Config
csv_path = "/content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100_Labels.csv"
image_root = "/content/drive/MyDrive/DM daisee/Train_Engagement_Sample_100"
checkpoint_dir = "/content/drive/MyDrive/DM daisee/checkpoints"
embedding_path = "/content/drive/MyDrive/DM daisee/embeddings_multi_output.pt"
os.makedirs(checkpoint_dir, exist_ok=True)

#  Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset
class DAiSEEMultiLabelDataset(Dataset):
    def __init__(self, csv_path, image_root, transform=None):
        self.data = pd.read_csv(csv_path)
        self.data.columns = self.data.columns.str.strip()
        self.image_root = image_root
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        clip_id = row['ClipID']
        folder, image = clip_id.split('_', 1)
        img_path = os.path.join(self.image_root, folder, image)

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        labels = torch.tensor([
            row['Boredom'],
            row['Engagement'],
            row['Confusion'],
            row['Frustration']
        ], dtype=torch.long)

        return image, labels

dataset = DAiSEEMultiLabelDataset(csv_path, image_root, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Model Definition
class MultiHeadResNet(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        self.heads = nn.ModuleList([nn.Linear(512, 4) for _ in range(4)])

    def forward(self, x):
        x = self.base(x)
        return [head(x) for head in self.heads]

# . Model, Optimizer, Loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()

model = MultiHeadResNet(resnet).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

#  Training Loop
epochs = 10

for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0
    correct = [0, 0, 0, 0]
    total = [0, 0, 0, 0]

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)

        loss = sum([criterion(out, labels[:, i]) for i, out in enumerate(outputs)])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        for i in range(4):
            preds = outputs[i].argmax(dim=1)
            correct[i] += (preds == labels[:, i]).sum().item()
            total[i] += labels.size(0)

    # Epoch results
    print(f"\nEpoch {epoch}/{epochs}")
    print(f"Loss: {total_loss / len(loader):.4f}")
    for i, name in enumerate(['Boredom', 'Engagement', 'Confusion', 'Frustration']):
        acc = 100 * correct[i] / total[i]
        print(f"{name} Accuracy: {acc:.2f}%")

    # Save model
    model_path = os.path.join(checkpoint_dir, f"resnet_multi_epoch{epoch}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"Saved model: {model_path}")

# Save Final Embeddings
model.eval()
embedding_list = []
label_list = []

with torch.no_grad():
    for imgs, labels in loader:
        imgs = imgs.to(device)
        features = model.base(imgs)  # [batch, 512]
        embedding_list.append(features.cpu())
        label_list.append(labels)

all_embeddings = torch.cat(embedding_list)
all_labels = torch.cat(label_list)

torch.save({
    'embeddings': all_embeddings,
    'labels': all_labels
}, embedding_path)

print(f"\nFinal embeddings saved to: {embedding_path}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/10
Loss: 3.9040
Boredom Accuracy: 70.40%
Engagement Accuracy: 68.90%
Confusion Accuracy: 62.20%
Frustration Accuracy: 59.50%
Saved model: /content/drive/MyDrive/DM daisee/checkpoints/resnet_multi_epoch1.pt

Epoch 2/10
Loss: 2.1188
Boredom Accuracy: 79.40%
Engagement Accuracy: 80.00%
Confusion Accuracy: 75.70%
Frustration Accuracy: 84.70%
Saved model: /content/drive/MyDrive/DM daisee/checkpoints/resnet_multi_epoch2.pt

Epoch 3/10
Loss: 1.5848
Boredom Accuracy: 85.00%
Engagement Accuracy: 85.80%
Confusion Accuracy: 81.80%
Frustration Accuracy: 89.80%
Saved model: /content/drive/MyDrive/DM daisee/checkpoints/resnet_multi_epoch3.pt

Epoch 4/10
Loss: 1.1264
Boredom Accuracy: 90.30%
Engagement Accuracy: 90.80%
Confusion Accuracy: 88.40%
Frustration Accuracy: 93.30%
Saved model: /content/drive/MyDrive/DM daisee/checkpoints/resnet_multi_epoch4.pt

Epoch 5/10
Loss: 0.7455
Boredom Accuracy: 95.40%
Engagement Accuracy: 96.10%
Confusion Accuracy: 93.70%
Frustration Accuracy: 96.50%
Saved 

In [ ]:
import torch

model_path = "/content/drive/MyDrive/DM daisee/checkpoints/resnet_multi_epoch10.pt"

# Load model weights
state_dict = torch.load(model_path, map_location='cpu')

# Print a few keys and shapes
for i, (k, v) in enumerate(state_dict.items()):
    print(f"{k}: {v.shape}")
    if i >= 10:
        break


base.conv1.weight: torch.Size([64, 3, 7, 7])
base.bn1.weight: torch.Size([64])
base.bn1.bias: torch.Size([64])
base.bn1.running_mean: torch.Size([64])
base.bn1.running_var: torch.Size([64])
base.bn1.num_batches_tracked: torch.Size([])
base.layer1.0.conv1.weight: torch.Size([64, 64, 3, 3])
base.layer1.0.bn1.weight: torch.Size([64])
base.layer1.0.bn1.bias: torch.Size([64])
base.layer1.0.bn1.running_mean: torch.Size([64])
base.layer1.0.bn1.running_var: torch.Size([64])


Validation

In [ ]:
class DAiSEEValidationDataset(Dataset):
    def __init__(self, label_csv, image_root, transform=None):
        self.df = pd.read_csv(label_csv)
        self.df.columns = self.df.columns.str.strip()
        self.image_root = image_root
        self.transform = transform

        # Match only clips in the Validation folder that exist
        self.valid_clips = []
        for _, row in self.df.iterrows():
            clip_id = row["ClipID"].replace(".avi", "")
            group_id = clip_id[:6]
            img_path = os.path.join(image_root, group_id, clip_id, "image_000.jpg")
            if os.path.exists(img_path):
                self.valid_clips.append((img_path, row))

    def __len__(self):
        return len(self.valid_clips)

    def __getitem__(self, idx):
        img_path, row = self.valid_clips[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        labels = torch.tensor([
            row['Boredom'],
            row['Engagement'],
            row['Confusion'],
            row['Frustration']
        ], dtype=torch.long)
        return img, labels


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_csv = "/content/drive/MyDrive/DM daisee/ValidationLabels.csv"
val_root = "/content/drive/MyDrive/DM daisee/DAiSEE_CroppedFaces/Validation"

val_dataset = DAiSEEValidationDataset(val_csv, val_root, transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
print(f" Found {len(val_dataset)} validation samples.")

✅ Found 1429 validation samples.


In [ ]:
from torchvision import models
import torch.nn as nn

class MultiHeadResNet(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        self.heads = nn.ModuleList([nn.Linear(512, 4) for _ in range(4)])

    def forward(self, x):
        x = self.base(x)
        return [head(x) for head in self.heads]

resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Identity()
model = MultiHeadResNet(resnet)

model_path = "/content/drive/MyDrive/DM daisee/checkpoints/resnet_multi_epoch10.pt"
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MultiHeadResNet(
  (base): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [ ]:
correct = [0, 0, 0, 0]
total = [0, 0, 0, 0]

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs)
        for i in range(4):
            preds = torch.argmax(outputs[i], dim=1)
            correct[i] += (preds == labels[:, i]).sum().item()
            total[i] += labels.size(0)

print("\n Final Validation Accuracy:")
for i, name in enumerate(['Boredom', 'Engagement', 'Confusion', 'Frustration']):
    acc = 100 * correct[i] / total[i]
    print(f"{name}: {acc:.2f}%")



 Final Validation Accuracy:
Boredom: 35.48%
Engagement: 47.52%
Confusion: 61.72%
Frustration: 71.66%
